In [1]:
import numpy as np
from PIL import Image
import time

In [6]:
# Chaotic map functions
def de_jong_iterate(x, y, a, b, c, d):
    x_next = np.sin(a * y) - np.cos(b * x)
    y_next = np.sin(c * x) - np.cos(d * y)
    return x_next, y_next

def generate_de_jong_sequence(x0, y0, a, b, c, d, num_iterations):
    x, y = x0, y0
    x_list, y_list = [], []
    for _ in range(num_iterations):
        x, y = de_jong_iterate(x, y, a, b, c, d)
        x_list.append(x)
        y_list.append(y)
    return np.array(x_list), np.array(y_list)

def van_der_pol_iterate(x, v, h, mu):
    x_next = x + h * v
    v_next = v + h * (mu * (1 - x**2) * v - x)
    return x_next, v_next

def generate_van_der_pol_sequence(x0, v0, h, mu, num_iterations):
    x, v = x0, v0
    x_list = []
    for _ in range(num_iterations):
        x, v = van_der_pol_iterate(x, v, h, mu)
        x_list.append(x)
    return np.array(x_list)

# Bit manipulation functions
def apply_xor_sequence(p, controls, targets, reverse=False):
    bits = [(p >> i) & 1 for i in range(8)]
    pairs = zip(controls[::-1], targets[::-1]) if reverse else zip(controls, targets)
    for c, t in pairs:
        bits[t] ^= bits[c]
    return sum(bit << i for i, bit in enumerate(bits))

def left_shift(p, k):
    return ((p << k) & 255) | (p >> (8 - k))

def right_shift(p, k):
    return (p >> k) | ((p << (8 - k)) & 255)

# Encryption and Decryption functions
def encrypt_image(image, x_conf, y_conf, x_block, x_shuffle, S):
    encrypted = image.copy()
    num_pixels = 256 * 256
    index = 0

    # Phase 1: Confusion - Chaotic Qubit-Level Transformation
    for y in range(256):
        for x in range(256):
            p = encrypted[y, x]
            controls = [int(2 * (x_conf[index + k] + 2)) % 8 for k in range(8)]
            targets = [int(2 * (y_conf[index + k] + 2)) % 8 for k in range(8)]
            encrypted[y, x] = apply_xor_sequence(p, controls, targets)
            index += 8

    # Phase 1: Diffusion - Block-Wise Geometric Transformations
    block_index = 0
    for by in range(32):
        for bx in range(32):
            block = encrypted[by*8:(by+1)*8, bx*8:(bx+1)*8].copy()
            select = int(3 * (x_block[block_index] + 2) / 4) % 3
            if select == 0:
                block = np.flip(block, axis=0)  # Vertical flip
            elif select == 1:
                block = np.flip(block, axis=1)  # Horizontal flip
            else:
                block = block.T  # Coordinate swap
            encrypted[by*8:(by+1)*8, bx*8:(bx+1)*8] = block
            block_index += 1

    # Phase 2: Confusion - Chaotic Selective Shuffle
    pixel_index = 0
    for y in range(256):
        for x in range(256):
            p = encrypted[y, x]
            select = int(3 * (x_shuffle[pixel_index] + 2) / 4) % 3
            if select == 0:
                p = left_shift(p, 2)
            elif select == 1:
                p = right_shift(p, 2)
            else:
                p = left_shift(p, 4)
            encrypted[y, x] = p
            pixel_index += 1

    # Phase 2: Diffusion - Chaotic Seed Diffusion
    encrypted ^= S
    return encrypted

def decrypt_image(encrypted, x_conf, y_conf, x_block, x_shuffle, S):
    decrypted = encrypted.copy()

    # Undo Phase 2: Diffusion - Chaotic Seed Diffusion
    decrypted ^= S

    # Undo Phase 2: Confusion - Chaotic Selective Shuffle
    pixel_index = 0
    for y in range(256):
        for x in range(256):
            p = decrypted[y, x]
            select = int(3 * (x_shuffle[pixel_index] + 2) / 4) % 3
            if select == 0:
                p = right_shift(p, 2)  # Inverse of left shift 2
            elif select == 1:
                p = left_shift(p, 2)   # Inverse of right shift 2
            else:
                p = right_shift(p, 4)  # Inverse of left shift 4
            decrypted[y, x] = p
            pixel_index += 1

    # Undo Phase 1: Diffusion - Block-Wise Geometric Transformations
    block_index = 0
    for by in range(32):
        for bx in range(32):
            block = decrypted[by*8:(by+1)*8, bx*8:(bx+1)*8].copy()
            select = int(3 * (x_block[block_index] + 2) / 4) % 3
            if select == 0:
                block = np.flip(block, axis=0)
            elif select == 1:
                block = np.flip(block, axis=1)
            else:
                block = block.T
            decrypted[by*8:(by+1)*8, bx*8:(bx+1)*8] = block
            block_index += 1

    # Undo Phase 1: Confusion - Chaotic Qubit-Level Transformation
    index = 0
    for y in range(256):
        for x in range(256):
            p = decrypted[y, x]
            controls = [int(2 * (x_conf[index + k] + 2)) % 8 for k in range(8)]
            targets = [int(2 * (y_conf[index + k] + 2)) % 8 for k in range(8)]
            decrypted[y, x] = apply_xor_sequence(p, controls, targets, reverse=True)
            index += 8

    return decrypted


In [7]:
# Main execution
images = ['apple.png', 'baboon.png', 'cameraman.png', 'jupiter.png', 'medical.png']
results = {}

# Parameters
a, b, c, d = 1.645, 1.905, 0.320, 1.530  # De Jong parameters
h, mu = 0.1, 0.06                       # Van Der Pol parameters
x0_dejong, y0_dejong = 0.1, 0.1
x0_vdp, v0_vdp = 0.1, 0.1
num_pixels = 256 * 256
num_blocks = 1024

# Generate chaotic sequences
x_conf, y_conf = generate_de_jong_sequence(x0_dejong, y0_dejong, a, b, c, d, 8 * num_pixels)
x_block, _ = generate_de_jong_sequence(x_conf[-1], y_conf[-1], a, b, c, d, num_blocks)
x_shuffle, _ = generate_de_jong_sequence(x_block[-1], y_conf[-1], a, b, c, d, num_pixels)
x_vdp = generate_van_der_pol_sequence(x0_vdp, v0_vdp, h, mu, num_pixels)
S = ((x_vdp - np.floor(x_vdp)) * 256).astype(np.uint8).reshape(256, 256)

for img_path in images:
    # Load and preprocess image
    img = Image.open(img_path).convert('L')
    img = img.resize((256, 256))
    image = np.array(img, dtype=np.uint8)

    # Encryption
    start_time = time.time()
    encrypted = encrypt_image(image, x_conf, y_conf, x_block, x_shuffle, S)
    enc_time = time.time() - start_time

    # Decryption
    start_time = time.time()
    decrypted = decrypt_image(encrypted, x_conf, y_conf, x_block, x_shuffle, S)
    dec_time = time.time() - start_time

    results[img_path] = {'enc_time': enc_time, 'dec_time': dec_time}

# Output results
print("Encryption and Decryption Times:")
for img, times in results.items():
    print(f"{img}: Encryption = {times['enc_time']:.4f}s, Decryption = {times['dec_time']:.4f}s")


Encryption and Decryption Times:
apple.png: Encryption = 0.9400s, Decryption = 0.9475s
baboon.png: Encryption = 1.8160s, Decryption = 2.5303s
cameraman.png: Encryption = 2.1994s, Decryption = 1.1593s
jupiter.png: Encryption = 0.9568s, Decryption = 0.9943s
medical.png: Encryption = 1.2570s, Decryption = 1.1705s
